In [1]:
import numpy as np
import pandas as pd
import os

# from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# from scipy.special import rel_entr
# from scipy.stats import entropy
# import scipy.stats
import math
import warnings 
warnings.filterwarnings("ignore") 

from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
! nvidia-smi

Failed to initialize NVML: Driver/library version mismatch


# MongoDB

In [5]:
from pymongo import MongoClient
import pymongo

In [6]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn

    db_rm_cn_2019 = db.rm_patent_cn_2019
    db_rm_us_2019 = db.rm_patent_us_2019
    db_rm_de_2019 = db.rm_patent_de_2019

    
except errors.ConnectionFailure as err:
    print(err)

# 2019

## Company數量

In [7]:
data_assignee = db_rm_de_2019.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [8]:
df_family_de = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [9]:
data_assignee = db_rm_cn_2019.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_cn = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [10]:
data_assignee = db_rm_us_2019.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_us = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [11]:
final_df = pd.concat([df_family_de,df_family_cn,df_family_us],axis=0)

In [12]:
# 取總數前30家公司
final_df.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(30)

,company,count
3315,Qualcomm Inc,1229
3523,Samsung Electronics Co Ltd,810
2527,LG Electronics Inc,750
2172,International Business Machines Corp,579
1985,Huawei Technologies Co Ltd,570
4506,Telefonaktiebolaget LM Ericsson AB,469
2160,Intel Corp,318
905,Cisco Technology Inc,263
2730,Microsoft Technology Licensing LLC,228
221,Apple Inc,228


In [13]:
com_df = final_df.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(30)
com_df.reset_index(drop=True, inplace=True)

In [14]:
com_df["count_log"] = com_df["count"].apply(lambda x: np.log(x))
com_df

,company,count,count_log
0,Qualcomm Inc,1229,7.113956
1,Samsung Electronics Co Ltd,810,6.697034
2,LG Electronics Inc,750,6.620073
3,International Business Machines Corp,579,6.361302
4,Huawei Technologies Co Ltd,570,6.345636
5,Telefonaktiebolaget LM Ericsson AB,469,6.150603
6,Intel Corp,318,5.762051
7,Cisco Technology Inc,263,5.572154
8,Microsoft Technology Licensing LLC,228,5.429346
9,Apple Inc,228,5.429346


公司名稱正規化

In [15]:
com_df["company"] = com_df.company.apply(lambda x: x.replace(" Inc","")\
                                                    .replace(" Co Ltd","")\
                                                    .replace(" Corp","")\
                                                    .replace("International Business Machines","IBM")\
                                                    .replace("Nippon Telegraph and Telephone","Nippon Telegraph & Tel")\
                                                    .replace("Alibaba Group Holding Ltd","Alibaba"))

In [16]:
com_df

,company,count,count_log
0,Qualcomm,1229,7.113956
1,Samsung Electronics,810,6.697034
2,LG Electronics,750,6.620073
3,IBM,579,6.361302
4,Huawei Technologies,570,6.345636
5,Telefonaktiebolaget LM Ericsson AB,469,6.150603
6,Intel,318,5.762051
7,Cisco Technology,263,5.572154
8,Microsoft Technology Licensing LLC,228,5.429346
9,Apple,228,5.429346


In [17]:
com_ls = list(com_df.company)

# Data 2019

富比士2000資料

In [18]:
f_data = pd.read_csv("./Forbes/Forbes_2019.csv")

In [19]:
f_data[f_data["Industry"]=="Semiconductors"]

,Company,Market Value,Revenue,Profits,Assets,Rank,Sector,Industry,Continent,Country,Headquarters,State,CEO,Forbes Webpage,Profits as % of Assets,Profits as % of Revenue
12,Samsung Electronics,272.424,221.506,39.882,304.138,13,Information Technology,Semiconductors,Asia,South Korea,South Korea,NaN,Hyun-Suk Kim,http://www.forbes.com/companies/samsung-electr...,0.131131,0.180049
43,Intel,263.127,70.848,21.053,127.963,44,Information Technology,Semiconductors,North America,United States,California,California,Robert Swan,http://www.forbes.com/companies/intel/,0.164524,0.297157
127,Taiwan Semiconductor,222.396,34.203,11.644,68.000,126,Information Technology,Semiconductors,Asia,Taiwan,Taiwan,NaN,Che Wei,http://www.forbes.com/companies/taiwan-semicon...,0.171235,0.340438
177,Broadcom,126.125,21.310,6.500,72.110,178,Information Technology,Semiconductors,North America,United States,California,California,Hock E. Tan,http://www.forbes.com/companies/broadcom/,0.090140,0.305021
178,SK Hynix,47.645,36.751,14.121,57.052,179,Information Technology,Semiconductors,Asia,South Korea,South Korea,NaN,Seok-Hee Lee,http://www.forbes.com/companies/sk-hynix/,0.247511,0.384234
205,Micron Technology,48.030,29.985,13.060,47.487,206,Information Technology,Semiconductors,North America,United States,Idaho,Idaho,Sanjay Mehrotra,http://www.forbes.com/companies/micron-technol...,0.275023,0.435551
309,Qualcomm,96.692,21.505,2.157,34.246,310,Information Technology,Semiconductors,North America,United States,California,California,Steve Mollenkopf,http://www.forbes.com/companies/qualcomm/,0.062985,0.100302
392,ASML Holding,85.551,12.911,2.979,24.779,393,Information Technology,Semiconductors,Europe,Netherlands,Netherlands,NaN,Peter Wennink,http://www.forbes.com/companies/asml-holding/,0.120223,0.230733
401,Texas Instruments,108.420,15.784,5.537,17.137,402,Information Technology,Semiconductors,North America,United States,Texas,Texas,Richard K. Templeton,http://www.forbes.com/companies/texas-instrume...,0.323102,0.350798
434,Applied Materials,41.726,16.802,3.949,18.922,435,Information Technology,Semiconductors,North America,United States,California,California,Gary Dickerson,http://www.forbes.com/companies/applied-materi...,0.208699,0.235032


In [20]:
Ind_ls = list(f_data[f_data["Industry"]=="Semiconductors"].Company)

## 使用company top 30 做join

In [21]:
forb_ls = list(np.where(com_df.company.isin(Ind_ls), 1, 0))

In [25]:
# pbc 0.09,0.09
comp_ls = ["Qualcomm","LG Electronics","IBM","Intel","Huawei Technologies","Telefonaktiebolaget LM Ericsson AB"]

In [26]:
competitor_ls = list(np.where(com_df.company.isin(comp_ls), 1, 0))

In [27]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.045937079942721074
ari:  0.135530457322718


## 使用Forbes做join

In [28]:
forb_ls = list(np.where(f_data.Company.isin(Ind_ls), 1, 0))

In [29]:
# pbc 0.09,0.09
comp_ls = ["Qualcomm","LG Electronics","IBM","Intel","Huawei Technologies","Telefonaktiebolaget LM Ericsson AB"]

In [30]:
competitor_ls = list(np.where(f_data.Company.isin(comp_ls), 1, 0))

In [31]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.06198979853295934
ari:  0.11277126875637866
